In [3]:
from stock_prediction.data_prep import Data_Prep
from stock_prediction.features_exo import exo_selection

In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import os
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error as MAPE
import itertools
import pdb

<ipython-input-4-870a85ae3975>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [21]:
# instantiate the Class 
vinci = Data_Prep('sap', 40)

In [22]:
# load data 
data_global = vinci.data_prep()

In [23]:
data_global.head()

,Date,Return_SAP.DE,Log_Return_SAP.DE,High-Low_SAP.DE,High-Close_SAP.DE,Low-Close_SAP.DE,Volume-Change_SAP.DE,Period_Volum_SAP.DE,Annual_Vol_SAP.DE,Period_Vol_SAP.DE,Return_stoxx_50,Period_Volum_stoxx_50,SAP.DE_relatif
0,2017-06-01,-0.001257,-0.001258,0.008938,0.006609,0.002313,-0.343074,-0.153886,0.163526,0.093153,0.003497,-0.115837,-0.004754
1,2017-06-02,0.008183,0.008150,0.006689,0.002289,0.004390,0.174486,-0.007768,0.163336,0.094545,0.006953,-0.103143,0.001230
2,2017-06-05,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.163214,0.092669,0.000000,0.000000,0.000000
3,2017-06-06,-0.004683,-0.004694,0.014419,0.007632,0.006736,inf,0.144818,0.161433,0.093277,-0.010479,-0.135073,0.005797
4,2017-06-07,-0.004077,-0.004086,0.008300,0.007453,0.000841,-0.200542,-0.085207,0.159945,0.094077,-0.001502,0.130258,-0.002575


In [24]:
# select fetarures 
data_exo = vinci.select_features(data_global, Return = True, Log_Return=False, High_Low=False, High_Close=True, Low_Close=True,
                        Volume_Change=False, Period_Volum=False, Annual_Vol=False,
                        Period_Vol=False, Return_Index=False, Volum_Index=False, Relative_Return=False)


In [25]:
data_exo

,Date,Return_SAP.DE,High-Close_SAP.DE,Low-Close_SAP.DE
0,2017-06-01,-0.001257,0.006609,0.002313
1,2017-06-02,0.008183,0.002289,0.004390
2,2017-06-05,0.000000,0.000000,0.000000
3,2017-06-06,-0.004683,0.007632,0.006736
4,2017-06-07,-0.004077,0.007453,0.000841
...,...,...,...,...
1009,2021-05-31,-0.013899,0.012156,0.000000
1010,2021-06-01,0.003524,0.019487,0.000000
1011,2021-06-02,0.001756,0.005959,0.006349
1012,2021-06-03,-0.005433,0.004581,0.008530


In [17]:
# add exogenous features 
#data_exo = exo_selection(data_exo, sp500=True, gold=True, eurusd=True, nasdaq=True, crude=True, vix=True)

In [26]:
# select first year 
data_exo = data_exo[-258:]

In [28]:
# rebase the indices 
data_exo = vinci.Price_Rebase(data_exo)

In [30]:
data_exo.columns

Index(['Date', 'High-Close_SAP.DE', 'Low-Close_SAP.DE', 'Return_SAP.DE_R'], dtype='object')

In [31]:
# assign the exogenous features 
y_endo = data_exo['Return_SAP.DE_R']
y_exo = np.array(data_exo[['High-Close_SAP.DE', 'Low-Close_SAP.DE']])

In [32]:
model_results_max = []


# we search all the possibilities for max_train 
    
for max_train in range(46,47,1) :

    splits = len(y_endo) - max_train

    # reset the lists for next for max_train
    mapeS = []
    base_score = []
    order = (0,1,0)
    # make the split of the indexes of our TS
    folds = TimeSeriesSplit(n_splits=splits, max_train_size=max_train, test_size=1)
    # fold_idx is the number of the split -- n_splits ?  
    # train_idx are the indexes of the split n°X
    # same for test_idx

    for train_idx, test_idx in folds.split(y_endo):

        # we retrieve the real data in y 
        #corresponding of the indexes splited 
        y_train = y_endo[train_idx]
        y_exo_train = y_exo[train_idx]
        y_test = y_endo[test_idx]
        y_exo_test = y_exo[test_idx]
        y_true = y_endo[test_idx]
        base = y_train.iloc[-1]
        
        # fit our model on y_train of the split n°X
        model = ARIMA(endog=y_train, exog=y_exo_train, order=order).fit()
        # we find our y_pred 
        y_pred = model.forecast(steps=len(y_test), exog=y_exo_test)[0]
        #pdb.set_trace()
        mapeS.append(MAPE(y_pred, y_true))
        # need np.array to compute mean
        base_score.append(MAPE(np.array([base]), y_true))
        mapeS_m = np.array(mapeS)
        base_score_m = np.array(base_score)
        # we store all the results 

        model_results_max.append([max_train, mapeS_m.mean(), base_score_m.mean()])

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/ari

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

...

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

In [33]:
model_results_max = pd.DataFrame(model_results_max, columns = ['max_train', 'MAPE', 'MAPE_base'])
model_results_max.head()

,max_train,MAPE,MAPE_base
0,46,0.013507,0.016141
1,46,0.011277,0.010583
2,46,0.008189,0.008918
3,46,0.006963,0.010349
4,46,0.008305,0.011602


In [67]:
#model_results_max.sort_values('MAPE').groupby('max_train').mean()[['MAPE', 'MAPE_base']].sort_values()
model_results_max.groupby('max_train').mean()[['MAPE', 'MAPE_base']].sort_values(by='MAPE').idxmin()[0]




46

In [18]:
# we show the best max_train regarding our MAPE
model_results_max.head().sort_values(by='MAPE')

,max_train,MAPE,MAPE_base
1,10,0.006039,0.006884
3,10,0.006552,0.008744
2,10,0.007005,0.007343
4,10,0.007060,0.011809
0,10,0.008630,0.003259


In [ ]:
model_results_max.